<a href="https://colab.research.google.com/github/emm32449/MCTS-in-Python/blob/main/Tic_Tac_Toe_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class GameFunctions:

    def initialize_board(self, num_pieces=2):

      # Start with an empty 3D board
      board = [[[' ']*3 for _ in range(3)] for _ in range(3)]
      return board

    def print_board(self, board):

        # Clear the previous board display
        self.output.clear_output(wait=True)

        # Create a list of strings for each depth
        board_strings = []
        for depth in range(len(board)):
            depth_string = f"Depth {depth}:\n"
            for row in board[depth]:
                row_string = '|'.join([' ' if cell == ' ' else cell for cell in row])
                depth_string += f"|{row_string}|\n"
            board_strings.append(depth_string)

        # Print the 3 depths side by side
        for i in range(len(board_strings[0].split('\n'))):
            print('   '.join(string.split('\n')[i] for string in board_strings))

    def check_win(self, board):
      # Check rows
      for depth in range(3):
          for row in range(3):
              if board[depth][row].count(board[depth][row][0]) == len(board[depth][row]) and board[depth][row][0] != ' ':
                  return board[depth][row][0]
      return None




    # Check if there's a draw!
    def check_draw(self, board):
      for depth in range(3):
          for row in range(3):
              for col in range(3):
                  if board[depth][row][col] == ' ':
                      return False
      return True

    def get_user_input(self, board):
        while True:
            user_input = input("Enter your move (depth[0-2] row[0-2] column[0-2]): ")
            try:
                move = tuple(map(int, user_input.split()))
                if len(move) != 3:
                    print("Invalid input. Enter three numbers.")
                    continue
                if any(coord not in [0, 1, 2] for coord in move):
                    print("Invalid input. Enter numbers between 0 and 2.")
                    continue
                if board[move[0]][move[1]][move[2]] != ' ':
                    print("Invalid move. The cell is not empty.")
                    continue
                return move
            except ValueError:
                print("Invalid input. Enter numbers.")

    def update_board(self, board, player_input, player):
        # Update the cell at the given position in the 3D board with the player's symbol
        board[player_input[0]][player_input[1]][player_input[2]] = player

    def switch_player(self, player):
        # Switch the current player
        return 'X' if player == 'O' else 'O'

    # Two Agents Play a Game
    def play_game(self, against_user=False):
        if against_user:
            board = self.initialize_board_user()
        else:
            board = self.initialize_board()
            current_player = 'X'

        while True:
            self.print_board(board)
            player_input = self.get_player_input(board, current_player)
            self.update_board(board, player_input, current_player)

            if self.check_win(board):
                print("Player {0} wins!".format(current_player))
                break
            elif self.check_draw(board):
                print("The game is a draw!")
                break

            current_player = self.switch_player(current_player)

    # Human Play Against an Agent
    def play_game(self, agent, against_user=False):
        if against_user:
            board = self.initialize_board_user()
        else:
            board = self.initialize_board()


        # board = self.initialize_board()  # Start a new game
        current_player = 'X'  # The first player is 'X'

        while True:
            self.print_board(board)
            if current_player == agent.player:
                action = agent.choose_action(board, self.get_available_actions(board))
            else:
                action = self.get_user_input(board)

            new_board, reward, done = self.make_move(board, action, current_player)
            board = new_board
            current_player = self.switch_player(current_player)

            if done:
                self.print_board(board)
                if self.check_win(board) == agent.player:
                    print("The agent wins!")
                elif self.check_draw(board):
                    print("The game is a draw!")
                else:
                    print("You win!")
                break

    # 2 Humans Play Each Other
    def play_game(self):
        board = self.initialize_board()
        current_player = 'X'  # The first player is 'X'

        while True:
            self.print_board(board)
            print(f"Player {current_player}, it's your turn.")
            action = self.get_user_input(board)
            self.update_board(board, action, current_player)

            if self.check_win(board):
                self.print_board(board)
                print(f"Player {current_player} wins!")
                break
            elif self.check_draw(board):
                self.print_board(board)
                print("The game is a draw!")
                break

            current_player = self.switch_player(current_player)

    def get_available_actions(self, board):
            available_actions = []

            for i in range(3):
                for j in range(3):
                    for k in range(3):
                        if board[i][j][k] == ' ':
                            available_actions.append((i, j, k))

            return available_actions

    def make_move(self, board, action, player):
            # Check if the action is valid
            if board[action[0]][action[1]][action[2]] != ' ':
                print("Invalid action!")
                return board, -1, True

            new_board = [[[cell for cell in row] for row in layer] for layer in board]  # Copy the game board
            new_board[action[0]][action[1]][action[2]] = player  # Make the move

            winner = self.check_win(new_board)
            game_over = winner is not None or self.check_draw(new_board)

            # Define a dictionary to map the game result to the reward
            rewards = {player: 1, None: 0, self.switch_player(player): -1}
            reward = rewards[winner] if game_over else 0

            return new_board, reward, game_over

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

class GameGUI:
    def __init__(self, game):
        self.game = game
        self.board = self.game.initialize_board()
        self.current_player = 'X'
        self.game_over = False
        self.output = widgets.Output()
        self.widget_output = widgets.Output()
        self.buttons = [[[None]*3 for _ in range(3)] for _ in range(3)]  # 3D list to store the buttons
        self.widget_output = widgets.Output()  # Output widget for the buttons
        self.text_output = widgets.Output()  # Separate Output widget for the text

        # Define the Buttons
        for depth in range(3):
            for row in range(3):
                for col in range(3):
                    # Create a button for each cell
                    self.buttons[depth][row][col] = widgets.Button(description=" ", layout=widgets.Layout(width='auto'),
                                                                   tooltip=f"({depth}, {row}, {col})")
                    self.buttons[depth][row][col].on_click(lambda btn,  depth=depth, row=row, col=col:
                                                           self.make_move(btn, depth, row, col))

    def display_board(self):
        # Create a GridBox for each depth
        grid_boxes = [widgets.GridBox([self.buttons[depth][i][j] for i in range(3) for j in range(3)],
                                      layout=widgets.Layout(grid_template_columns="repeat(3, 40px)")) for depth in range(3)]

        # Create labels for each depth
        labels = [widgets.Label(value=f"M: {depth}") for depth in range(3)]

# Display the labels and GridBoxes horizontally with spacing, and the Output widgets below the buttons
        display(widgets.VBox([widgets.HBox([widgets.VBox([labels[depth], grid_boxes[depth]],
                                        layout=widgets.Layout(border='1px solid grey', padding='5px')) for depth in range(3)],
                                        layout=widgets.Layout(justify_content='flex-start', padding='5px', margin='5px')),
                              self.widget_output,  # Display the Output widget for the buttons
                              self.text_output]))  # Display the separate Output widget for the text

    def print_board(self, board):
        # Clear the previous board display
        self.text_output.clear_output(wait=True)

        # Create a list of strings for each depth
        board_strings = []
        for depth in range(len(board)):
            depth_string = f"Depth {depth}:\n"
            for row in board[depth]:
                row_string = '|'.join([' ' if cell == ' ' else cell for cell in row])
                depth_string += f"|{row_string}|\n"
            board_strings.append(depth_string)

        # Print the 3 depths side by side
        with self.text_output:
            for i in range(len(board_strings[0].split('\n'))):
                print('   '.join(string.split('\n')[i] for string in board_strings))

    def make_move(self, btn, depth, row, col):
        if not self.game_over:
            action = (depth, row, col)
            # Check if the action is valid
            if self.board[action[0]][action[1]][action[2]] != ' ':
                with self.output:
                    print("Invalid action!")
                return

            self.board[action[0]][action[1]][action[2]] = self.current_player  # Make the move
            btn.description = self.current_player  # Update the button's description with the current player's symbol
            btn.disabled = True  # Disable the button after it's clicked

            winner = self.game.check_win(self.board)
            self.game_over = winner is not None or self.game.check_draw(self.board)

            if self.game_over:
                with self.output:
                    print(f"Player {self.current_player} wins!") if winner is not None else print("The game is a draw!")

            else:
                self.current_player = self.game.switch_player(self.current_player)  # Switch the current player


In [ ]:
# Create an instance of the GameFunctions class
game_functions = GameFunctions()

# Create an instance of the GameGUI class with the GameFunctions instance
gui = GameGUI(game_functions)
gui.display_board()
game_functions.play_game(gui)


TypeError: GameFunctions.play_game() takes 1 positional argument but 2 were given